# Churn Prediction Model Inference

## Inference with the Champion model

Models in Unity Catalog can be loaded for use in batch inference pipelines. Generated predictions would be used to advise on customer retention strategies, or be used for analytics. The model in use is the __@Champion__ model, and we will load it for use in our pipeline.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/advanced/banners/mlflow-uc-end-to-end-advanced-5.png?raw=true" width="1200">

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1832744760933926&notebook=%2F02-mlops-advanced%2F05_batch_inference&demo_name=mlops-end2end&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fmlops-end2end%2F02-mlops-advanced%2F05_batch_inference&version=1">

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-mlops-end2end-edgar_aguilerarod` from the dropdown menu ([open cluster configuration](https://dbc-07122dbb-1c85.cloud.databricks.com/#setting/clusters/0102-173414-9ev1v92w/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('mlops-end2end')` or re-install the demo: `dbdemos.install('mlops-end2end')`*

In [0]:
%pip install --quiet mlflow==2.14.3
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $adv_mlops=true $setup_adv_inference_data=true

##Deploying the model for batch inferences

<!--img style="float: right; margin-left: 20px" width="600" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/churn_batch_inference.gif" /-->

Now that our model is available in the Unity Catalog Model Registry, we can load it to compute our inferences and save them in a table to start building dashboards.

We will use the feature engineering client's `score_batch` method which would automatically load a pyspark UDF and distribute the inference on the entire cluster. If the data is small, we can also load the model in plain python and use a pandas Dataframe.

If you don't know how to start, you can get sample code from the __"Artifacts"__ page of the model's experiment run.

## Run inferences

### Batch inference on the Champion model

We are ready to run inference on the Champion model. We will leverage the feature engineering client's `score_batch` method and generate predictions for our customer records.

For simplicity, we assume that features have been pre-computed for all new customer records and already stored in a feature table. These are typically done by separate feature engineering pipelines.

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
import pyspark.sql.functions as F

# Load customer features to be scored
inference_df = spark.read.table("advanced_churn_cust_ids")

fe = FeatureEngineeringClient()

# Fully qualified model name
model_name = f"{catalog}.{db}.advanced_mlops_churn"

# Model URI
model_uri = f"models:/{model_name}@Champion"

# Batch score
preds_df = fe.score_batch(df=inference_df, model_uri=model_uri, result_type="string")
display(preds_df)


That's it! Our data can now be saved as a table and re-used by the Data Analyst / Marketing team to take special action and reduce Churn risk on these customers!

Your data will also be available within Genie to answer any churn-related question using plain text english!

### Save the predictions for monitoring

Since we want to monitor the model and its predictions over time, we will save the predictions, along with information on the model used to produce them. This table can then be monitored for feature drift and prediction drift.

Note that this table does not have the ground truth labels. These are usually collected and made available over time, and in many cases, may not even be available! However, this does not stop us from monitoring the data for drift, as that alone may be a sign that the model has to be retrained.

The table displayed below is saved into `advanced_churn_offline_inference`. It includes the model version used for scoring, the model alias, the predictions and timestamp when the inference was made. It does not contain any labels.


In [0]:
from mlflow import MlflowClient
from datetime import datetime
client = MlflowClient()

model = client.get_registered_model(name=model_name)
model_version = int(client.get_model_version_by_alias(name=model_name, alias="Champion").version)

In [0]:
import pyspark.sql.functions as F
from datetime import datetime, timedelta

offline_inference_df = preds_df.withColumn("model_name", F.lit(model_name)) \
                              .withColumn("model_version", F.lit(model_version)) \
                              .withColumn("model_alias", F.lit("Champion")) \
                              .withColumn("inference_timestamp", F.lit(datetime.now()- timedelta(days=2)))

offline_inference_df.write.mode("overwrite") \
                    .saveAsTable("advanced_churn_offline_inference")

display(offline_inference_df)

### Congratulations! You have successfully used the model for batch inference.

Let's look at how we can deploy this model as a REST API endpoint for real-time inference.

Next:  [Serve the features and model in real-time]($./06_serve_features_and_model)